In [13]:
## import all libraries

import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [8]:
## download train and test dataset with normalize

batch_size = 200 # size of one batch for train


train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,)) # standard normalize for mnist 
                       ])),
        batch_size=batch_size, shuffle=True)




test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=batch_size, shuffle=True)

In [9]:
# we use standard pytorch class nn.Module to create neural network

class LinearModel(nn.Module):
    def __init__(self):
        # initilization
        super().__init__()
        # neural network with 4 layers
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)


    def forward(self, x):
        # principles
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)

In [10]:
linModel = LinearModel()
print(linModel)

LinearModel(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)


In [12]:
# create SGD optimization
optimizer = optim.SGD(linModel.parameters(), lr=0.05, momentum=0.9) # wo momentum

# create loss func
criterion = nn.NLLLoss()

In [32]:
# train
for epoch in range(10):
   for batch_idx, (data, target) in enumerate(train_loader):
       data, target = Variable(data), Variable(target)
       data = data.view(-1, 28 * 28)
       optimizer.zero_grad()
       no = linModel(data)
       loss = criterion(no, target)
       loss.backward()
       optimizer.step()
       if batch_idx % 10 == 0:
          print("Epoch №:", epoch, ", percent of batch:", float('{:.0f}'.format(((100. * batch_idx) / len(train_loader)))),
                ", Loss: ", float('{:.6f}'.format(loss.item())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch №: 0 , percent of batch: 0.0 , Loss:  2.9e-05
Epoch №: 0 , percent of batch: 3.0 , Loss:  7.6e-05
Epoch №: 0 , percent of batch: 7.0 , Loss:  4.5e-05
Epoch №: 0 , percent of batch: 10.0 , Loss:  5.4e-05
Epoch №: 0 , percent of batch: 13.0 , Loss:  6.9e-05
Epoch №: 0 , percent of batch: 17.0 , Loss:  0.000172
Epoch №: 0 , percent of batch: 20.0 , Loss:  7.5e-05
Epoch №: 0 , percent of batch: 23.0 , Loss:  6e-05
Epoch №: 0 , percent of batch: 27.0 , Loss:  5.8e-05
Epoch №: 0 , percent of batch: 30.0 , Loss:  6.8e-05
Epoch №: 0 , percent of batch: 33.0 , Loss:  4.8e-05
Epoch №: 0 , percent of batch: 37.0 , Loss:  7.4e-05
Epoch №: 0 , percent of batch: 40.0 , Loss:  3.5e-05
Epoch №: 0 , percent of batch: 43.0 , Loss:  3.5e-05
Epoch №: 0 , percent of batch: 47.0 , Loss:  4.7e-05
Epoch №: 0 , percent of batch: 50.0 , Loss:  5.2e-05
Epoch №: 0 , percent of batch: 53.0 , Loss:  4.9e-05
Epoch №: 0 , percent of batch: 57.0 , Loss:  3e-05
Epoch №: 0 , percent of batch: 60.0 , Loss:  5.3e-05

In [33]:
# now we will test our neural network on train dataset

final_loss = 0
answ = 0
for data, target in test_loader:
   data, target = Variable(data, volatile=True), Variable(target)
   data = data.view(-1, 28 * 28)
   no = linModel(data)
   final_loss += criterion(no, target).item()
   last = no.data.max(1)[1]
   answ += last.eq(target.data).sum()

final_loss /= len(test_loader.dataset)



print("Average loss:", float('{:.6f}'.format(final_loss)), ", accuracy:", float('{:.3f}'.format((100. * correct / len(test_loader.dataset)))))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Average loss: 0.000447 , accuracy: 98.36


In [34]:
# and now we will create convolutional neural network for our MNIST dataset

class ConvNet(nn.Module): 
    def __init__(self): 
        # initilization
        super().__init__()
        # first layer
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2), 
            nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
        # second layer
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), 
            nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
        # drop out
        self.drop_out = nn.Dropout() 
        # two fully connected layers
        self.fc1 = nn.Linear(7 * 7 * 64, 1000) 
        self.fc2 = nn.Linear(1000, 10)


    def forward(self, x):
        # forward function for our layers 
        out = self.layer1(x) 
        out = self.layer2(out) 
        out = out.reshape(out.size(0), -1) 
        out = self.drop_out(out) 
        out = self.fc1(out) 
        out = self.fc2(out) 
        return out

In [38]:
ConvModel = ConvNet()

# create Adam optimization
optimizer = torch.optim.Adam(ConvModel.parameters(), lr=0.001)

# create loss func
criterion = nn.CrossEntropyLoss()

In [41]:
total_step = len(train_loader)
all_losses = []
all_acc = []
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # forward step
        no = ConvModel(images)
        loss = criterion(no, labels)
        all_losses.append(loss.item())

        # back step and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # accuracy
        sum = labels.size(0)
        local, expected = torch.max(no.data, 1)
        correct = (expected == labels).sum().item()
        all_acc.append(correct / sum)

        if (i + 1) % 100 == 0:
            print("Epoch №:", epoch + 1, ", percent of batch", float('{:.2f}'.format((i + 1) / 300)) * 100,
                  ", loss:", loss.item(), "accuracy:", float('{:.2f}'.format((correct / sum) * 100)))

Epoch №: 1 , percent of batch 33.0 , loss: 0.03571074828505516 accuracy: 98.5
Epoch №: 1 , percent of batch 67.0 , loss: 0.015923863276839256 accuracy: 99.5
Epoch №: 1 , percent of batch 100.0 , loss: 0.06652771681547165 accuracy: 97.5
Epoch №: 2 , percent of batch 33.0 , loss: 0.05505935847759247 accuracy: 98.5
Epoch №: 2 , percent of batch 67.0 , loss: 0.04607236385345459 accuracy: 99.5
Epoch №: 2 , percent of batch 100.0 , loss: 0.043576039373874664 accuracy: 98.0
Epoch №: 3 , percent of batch 33.0 , loss: 0.04587356001138687 accuracy: 98.0
Epoch №: 3 , percent of batch 67.0 , loss: 0.02097713015973568 accuracy: 99.0
Epoch №: 3 , percent of batch 100.0 , loss: 0.0203156229108572 accuracy: 99.5
Epoch №: 4 , percent of batch 33.0 , loss: 0.024467742070555687 accuracy: 99.5
Epoch №: 4 , percent of batch 67.0 , loss: 0.05455690994858742 accuracy: 98.5
Epoch №: 4 , percent of batch 100.0 , loss: 0.006874406710267067 accuracy: 100.0
Epoch №: 5 , percent of batch 33.0 , loss: 0.07815394550